# Calling C (and Fortran) from Julia

Somewhat like `ctypes` and similar libraries in Python, Julia has a built-in `ccall` feature to call functions in external compiled (C ABI) libraries.

In [ ]:
ccall(:printf, Cint, (Ptr{UInt8},), "Hello, world!")

The format is `ccall(function name, return type, argument types, arguments...)`.

You can also call functions in arbitrary shared libraries / DLLs:

In [ ]:
mysin(x) = ccall((:sin, "libm"), Cdouble, (Cdouble,), x)

In [ ]:
mysin(3.0) - sin(3.0)

In [ ]:
mysin(3) # note that Julia automatically converts types as necessary

Unlike Python, however, Julia's speed means that it is perfectly fine to call C functions operating on small data, like single numbers — you don't have to "vectorize" on the C side first, and you can instead vectorize on the Julia side.

In [ ]:
@vectorize_1arg Real mysin

In [ ]:
mysin([1,2,3,4])

In [ ]:
code_native(mysin, (Float64,))

# Calling Python from Julia

Thanks to a package called `PyCall`, Julia **call arbitrary Python** functions by calling directly into CPython's `libpython`:

In [ ]:
using PyCall

In [ ]:
@pyimport math as pymath

In [ ]:
pymath.cos(3)

## Low-level dissection of a Python call

Let's break this down.  When you run `pymath.sin(3)`, Julia:

* First converts `3` into the corresponding Python object via `PyObject(3)`.
* Then calls `pymath.sin` via the `libpython` routine `PyObject_Call`.
* Finally, detects the type of the return value and converts it back to a Julia object.

In terms of lower-level steps, it is doing:

In [ ]:
three = PyObject(3)   # calls PyInt_FromSsize_t in CPython library

In [ ]:
@pyimport math  # calls PyImport_AddModule in CPython

One slight annoyance is that Julia doesn't (yet) let you override `.`, so `foo.bar` in Python generally becomes `foo[:bar]` in Julia (or `foo["bar"]` if you want to leave the result as an unconverted Python object).  This will change in a future Julia release.

In [ ]:
sinfunc = math.sin   # calls PyObject_GetAttrString

In [ ]:
returnval = pycall(sinfunc, PyObject, three) # calls PyObject_Call

In [ ]:
convert(Float64, returnval)   # if we know the type we want, we can specify it
                              # calls PyFloat_AsDouble in CPython

In [ ]:
convert(PyAny, returnval)     # if we don't know the type we want, PyAny will detect it

## Copy-free sharing of data

PyCall allows large arrays and dictionaries to be passed **without making a copy**.

### Passing arrays

For example, **Julia arrays are wrapped by NumPy arrays** with **shared data**.

In [ ]:
A = rand(3,5)

In [ ]:
Apy = PyObject(A)

In [ ]:
A[1,1] = 17
Apy

In [ ]:
@pyimport numpy as np
x = [-100, 39, 59, 55, 20]
np.irr(x)

By default, PyCall makes a copy of arrays when converting from Python back to Julia:

In [ ]:
np.cumsum(x)

But we can specify a copy-free return if desired by calling the lower-level `pycall` function and specifying the desired return type as `PyArray`:

In [ ]:
xsum = pycall(np.cumsum, PyArray, x)

The resulting *NumPy* array can be passed to other Julia routines without making a copy:

In [ ]:
mean(xsum)

There is also a `PyVector` type that you can use to wrap **Python lists** without making a copy:

## Starting from Julia v0.4

In [ ]:
@pyimport sys

In [ ]:
sys.path

### Passing dictionaries

There is also a `PyDict` type that you can use to **share a dictionary** between Julia and Python.

In [ ]:
d = PyDict()

In [ ]:
d["hello"] = 7
d[23] = "goodbye"
d

For fun, we'll use `pyeval` to pass `d` as a local variable `dict` to an arbitrary string of Python code that we want to evaluate, in this case a list comprehension in Python:

In [ ]:
pyeval("[x for x in dict.keys()]", dict=d)

## Passing Julia functions to Python

Arbitrary Julia functions can be passed to Python.   They get converted into callable Python objects of a custom class, whose `__call__` method executes the Julia code:

In [ ]:
foo(x) = x + 1
pyfoo = PyObject(foo)

In [ ]:
pycall(pyfoo, PyAny, 17)

This is extremely useful for calling functions for optimization, root-finding, etcetera, from SciPy.  For example, let's solve a transcendental equation to find a root of $f(x) = \cos(x) - x$:

In [ ]:
@pyimport scipy.optimize as so
function f(x)
    println("   calling f($x)")
    cos(x) - x
end
so.newton(f, 3)

There is a bit of magic going on in passing Julia functions to Python.  To define a new Python type from the CPython API, we create a C `PyTypeObject` struct, and we need to stick a C function pointer into its `tp_call` slot to give it a `__call__` method.  

A *C function pointer is just the address of the compiled machine instructions,* and since Julia has these instructions from its JIT compiler it can give you the address of the instructions using an intrinsic called `cfunction`, e.g.

In [ ]:
cfunction(f, Float64, (Float64,))

This ability to get C function pointers from Julia functions is the key to calling **any C API expecting a callback routine**, not just Python.  See the blog post: [Passing Julia Callback Functions to C](http://julialang.org/blog/2013/05/callback/).

## Calling Matplotlib

To get Matplotlib working in IJulia, we had to do a bit more work, similar to what IPython had to do to get its `pylab` option working.   For GUI windows, we had to implement the GUI event loop for the Python GUI toolkit(s) (PyQt, Gtk, wx) in Julia.  For inline plots, we had to monkey-patch Matplotlib to intercept its drawing commands queue the figure for rendering as an image to be sent to the front-end.  All of this is done by the `PyPlot` Julia module:

In [ ]:
using PyPlot

In [ ]:
x = linspace(0,2π,1000)
plot(x, sin(3x + cos(5x)), "b--")
title("a funny plot")

In [ ]:
y = linspace(0,2π,50)
surf(y, y, sin(y) .* cos(y)')

Really, the **whole Matplotlib API** is available for use.  It has everything you might want (in 2d, at least), if you dig long enough through the manuals:

In [ ]:
xkcd()

fig = figure(figsize=(10,5))
ax = axes()
p = plot(x,sin(3x + cos(5x)))
ax[:set_xlim]([0.0,6])
annotate("A little\nsketchy",xy=[0.98,.001],arrowprops=Dict("arrowstyle"=>"->"),xytext=[1.3,-0.3])

xticks([])
yticks([])
xlabel("TIME")
ylabel("PRODUCTIVITY")
title("An xkcd-style plot in Julia")

ax[:spines]["top"][:set_color]("none") # Remove the top axis boundary
ax[:spines]["right"][:set_color]("none") # Remove the right axis boundary


## Calling scikit-image

On Wednesday, Stéfan van der Walt showed some [interesting demos using scikit-image](http://nbviewer.ipython.org/github/scikit-image/skimage-tutorials/blob/212c50a47f949070decd2047d2d4ff1c2114a5b3/2014-euroscipy/index.ipynb).  It is straightforward to try a couple of his examples from Julia:

In [ ]:
@pyimport skimage.data as skdata
image = skdata.camera()

In [ ]:
@pyimport skimage.exposure as exposure
subplot(1,3,1)
imshow(image, cmap="jet")
subplot(1,3,2)
imshow(image, cmap="gray")
subplot(1,3,3)
imshow(exposure.equalize_adapthist(image), cmap="gray")

In [ ]:
@pyimport skimage.filter as filters
threshold = filters.threshold_otsu(image)
println("Otsu threshold = $threshold")
imshow(image .> threshold, cmap="gray")